# <font color='red'>WORK SOLELY DONE BY THOMAS DANIEL JEAN MARCHEGAY AND THEO CHICHERY.</font>

# Problem Overview 

Objective : The aim of this project is to maximize the value of a biscuit roll.

Roller property : A size of 500 and defects of 3 types placed at different positions 

Cookie properties: There are four types of cookies, each with their own sizes, values and possible defects.

## Choice of Algorithms

## Informed Search

The A* algorithm is particularly well-suited to this complex problem due to its ability to find optimal solutions while minimizing computational costs. Here are some key reasons justifying its implementation:

Resource Optimization: A* excels in problems where the goal is to maximize a value while adhering to complex constraints, such as maximizing the value of cookies placed while respecting size and defect restrictions.

Powerful Heuristics: With well-designed heuristics, A* can effectively guide the search by prioritizing promising solutions. For example, a heuristic could estimate the maximum remaining value for the unused dough segment.

Constraint Management: A* allows for the integration of placement constraints for cookies, such as no overlap or defect thresholds, by exploring only valid solutions.

Minimizing Waste: The algorithm can account for the cost of material waste (penalties for unused areas) and search for solutions that minimize this waste.

Flexibility and Extensibility: A* can be easily adapted to include additional constraints or specific configurations, making it a robust choice for this multi-constraint problem.

Performance Comparison: Compared to other methods like exhaustive search or local approaches, A* combines the advantages of informed search and optimality, offering a balanced solution between performance and accuracy.

## Constraint Satisfaction Problem

Adapted to Constraints: The problem is defined by strict rules (overlap, defect thresholds, maximum length) which the CSP naturally handles.

Clear Modeling: Variables (positions, cookie types) and constraints (no overlap, threshold validity) are effectively formalized.

Optimized Exploration: CSP algorithms quickly eliminate invalid configurations and find valid solutions.

Possible Optimization: The addition of an objective function to maximize total value while minimizing penalties.

Flexibility: Easily adaptable to modifications or additions of new constraints/types of cookies.

## Dynamic Programming

Natural Decomposition of the Problem: The problem can be broken down into independent subproblems, such as maximizing the value of a given dough segment while respecting constraints.

Reuse of Solutions: Solutions to subproblems (e.g., the best configuration for a given dough length) can be memorized to avoid redundant calculations.

Global Optimization: DP guarantees an optimal solution by exploring all possible combinations while avoiding unnecessary explorations.

Constraint Management: Rules such as defect thresholds and non-overlap can be directly integrated into states or transitions.

Computational Efficiency: Compared to exhaustive search, DP significantly reduces computation time through memoization and guided exploration.

## Nature Inspired Algorithms

### Genetic Algorithm

Exploration in Complex Spaces: Genetic algorithms (GA) excel in large, complex solution spaces, such as arranging cookies on dough with varied constraints.

Adaptability to Constraints: GA can integrate constraints (like defect thresholds or no overlap) through fitness functions or repair mechanisms.

Effective Optimization: By exploring multiple solutions simultaneously, GA avoids local minima and increases the chances of finding near-optimal solutions.

Flexibility of Objectives: The fitness function can easily be modified to maximize profits while minimizing penalties.

Scalability: GA can handle more complex scenarios, such as changes in dough size or the addition of new cookie types.

### Ant Colony Optimization

Efficient Exploration of Solutions: ACO is particularly suited for complex combinatorial search problems like this one, where multiple solutions need to be explored to maximize cookie production.

Adaptation to Constraints: ACO can be configured to account for specific constraints (like no overlap or defect thresholds) by adjusting the solution construction rules and quality evaluations.

Distributed and Robust Approach: Just like ants explore multiple paths simultaneously, ACO allows for exploring various configurations in parallel, reducing the risk of falling into suboptimal solutions.

Objective Function Optimization: ACO naturally optimizes solutions by maximizing placed cookies while minimizing penalties associated with unused spaces.

Adaptability: ACO is flexible and can easily adjust to changes in problem parameters, such as new constraints or cookie types.

## Imports

In [11]:
import csv
import random
import numpy as np
from collections import defaultdict
import pandas as pd
import heapq
import math
import time
from ortools.sat.python import cp_model


# Classes creation 

The following class will be used for the Informed Search CSP and dynamic programming approach.

In [12]:
class Biscuit:
    def __init__(self, length, value, a,b,c,index):
        """The Biscuit class is used to create a biscuit with the following parameters:
        
        Parameters:
        - Biscuit_length: Size the Biscuit will take
        - Biscuit_value: Value of the biscuit
        - Possible_defaults_a: Maximum defects a on which the cookie can be placed
        - Possible_defaults_b: Maximum defects b on which the cookie can be placed
        - Possible_defaults_c: Maximum defects c on which the cookie can be placed
        - Index: Value with which the biscuit will be represented in the grid
        """
        self.length = length
        self.value = value
        self.a = a
        self.b = b
        self.c = c
        self.index = index

class Defect:
    def __init__(self,a=0,b=0,c=0):
        
        """The Defect class is used to create a Defect with the following parameters:
        
        Parameters:
        - a: Number of defects a
        - b: Number of defects b
        - c: Number of defects c
        """
        self.a = a
        self.b = b
        self.c = c

class Probleme_biscuit:
    def __init__(self, initial, Biscuits, error):
        
        """The cookie problem class is used to create the environment for the a_star method, which is initialized with the parameters :
        
        Parameters:
        - initial: Number of defects a
        - Biscuits: Tables with all the company's biscuits
        - error: A good-sized empty table
        """
        self.initial = initial
        self.goal = 0
        self.Biscuits = Biscuits
        self.error = error

    def actions(self, state, Biscuits, index):
        possible_b = []
        return check_possible_biscuits(state, Biscuits, index, self.error)

    def result(self, state, index, action):
        new_roll = state[:]
        for i in range(index, index + action.length):
            new_roll[i] = action.index
        return new_roll

    def goal_test(self, value):#The goal is reached when a roll is filled.
        return  get_index(value) >= 500

    def path_cost(self, c, state1, action, state2):
        return c + 1

    def value(self, state):
        raise NotImplementedError

    def cost(self, tab):
        return Count_value(tab)
    def unit(self, tab):
        return 0
    def h2(self,tab):
        s=0
        for i in range(get_index(tab),len(tab)):
            if (tab[i]==0):
                s=s+2.3
        return s+(Count_value(tab))+(Count_4(tab)*0.1)
        
        
        

# Problem implementation

Firstly, we decided to implement the errors by rounding down, as each cookie must be placed on whole positions and a cookie will take up an entire slot.

We also decided to put the errors in an array the size of the roll, for easy access to errors with a desired position.


In [13]:
defect=pd.read_csv("defects.csv")
defect = defect.sort_values(by='x')
defect2=defect.reset_index(drop=True)
defects=[0]*500


def fill_defect(defect,df):
    """The purpose of this function is to take an error dataframe and fill an array of size 500 with errors rounded down. With the following parameters:
        
        Parameters:
        - defect: An array large enough for the dataframe
        - Df: The error dataframe with a column x and class
    """
    a=0
    b=0
    c=0
    borne_sup=1
    
    for i in range (500): 
        if(df["x"][i]<=borne_sup):#If the error position is lower than the upper value, the corresponding error is added.
            if(df["class"][i]=="a"):
                a=a+1
            if(df["class"][i]=="b"):
                b=b+1
            if(df["class"][i]=="c"):
                c=c+1
        else :
            while(df["x"][i]>borne_sup):#As soon as the top position is moved, an error class is created with all the corresponding errors. Then we increase the upper position and start again. 
                defect[borne_sup-1]=Defect(a,b,c)
                a=0
                b=0
                c=0
                borne_sup=borne_sup+1
            if(df["class"][i]=="a"):
                a=a+1
            if(df["class"][i]=="b"):
                b=b+1
            if(df["class"][i]=="c"):
                c=c+1
    defect[borne_sup - 1] = Defect(a, b, c)
    return defect


error=fill_defect(defects,defect2)
    

To create the biscuits, we create the 4 biscuits of the problem in a table so that we can access all the biscuits directly.

In [4]:
B1=Biscuit(4,3,4,2,3,1)
B2=Biscuit(8,12,5,4,4,2)
B3=Biscuit(2,1,1,2,1,3)
B4=Biscuit(5,8,2,3,2,4)
Biscuits=[B4,B2,B1,B3]

First, we'll create functions to manage the addition of cookies. We'll need to create a function to add a cookie, which we'll name (add_biscuit), and a function to return the cookies possible at a given position (check_possible_biscuits). Once we've implemented these two functions, we'll have all the functions we need to manage the cookies

# Functions

In [5]:
def add_biscuit(tab, biscuit, index):
    """ adds the desired cookie to the desired position and returns the grid. It takes the following parameters : 

        Parameters:
        - tab: The current grid
        - biscuit: The list of all possible cookies
        - index: Position where a cookie is to be added
    
    """
    tab2=tab[:]
    if index + biscuit.length <= 500:
        for i in range(index, index + biscuit.length):
            tab2[i] = biscuit.index
    return tab2



def check_possible_biscuits(tab,biscuit,index,error):#Check all the cookies to see if they can be placed, and if so, add them to the list.

    """ Checks whether the biscuits can be placed in one position. It takes the following parameters : 

        Parameters:
        - tab: The current grid
        - biscuit: The list of all possible cookies
        - index: Position where a cookie is to be added
        - error: List containing all our errors
    """
    biscuits_poss=[]
    for bis in biscuit:
        a=0
        b=0
        c=0
        if(index+bis.length-1<500):
            for i in range(index,index+bis.length):
                a1=error[i].a
                b1=error[i].b
                c1=error[i].c
                a=a+a1
                b=b+b1
                c=c+c1
            if(a<=bis.a and b<=bis.b and c<=bis.c):#If the cookie does not exceed the errors, the cookie is added to the possible cookies.
                biscuits_poss.append(bis)
    return biscuits_poss

   

We now create two functions to manage the grid, a function that will allow us to retrieve the first empty position (get_index )and a function that will leave an empty space represented by a -1 at the first empty position of the grid.

In [6]:
def get_index(tab):
    #Returns the first empty position in the array
    for i in range(len(tab) - 1, 0, -1):
        if tab[i] != 0:
            return i + 1
    return 0
def add_empty(tab):#Add a cookie representing an empty space
    tab2=tab
    ind=get_index(tab)
    tab2[ind]=-1
    return tab2
    

Finally, functions to view the results of our grid are created, the first and most important being Count_value, which counts the total cost of the grid. We then create the function associated with the Count_4 heuristic, which allows us to browse a grid and returns the number of cookies 4 in the grid. Finally, we create a function that retrieves the best grid among several, based on total cost.

In [7]:
def Count_value(tab):#Count the grid value
    value=0
    i=0
    while(i<500):
        if(tab[i]==1):
            value=value+B1.value# We add the value of the cookie to the cost of the grid
            i=i+B1.length#We go to the position of the next cookie
        elif(tab[i]==2):
            value=value+B2.value
            i=i+B2.length
        elif(tab[i]==3):
            value=value+B3.value
            i=i+B3.length
        elif(tab[i]==4):
            value=value+B4.value
            i=i+B4.length
        elif(tab[i]<=0):
            value=value-1
            i=i+1
    return value
def best(tab):#Takes  as parameters a list of grids and returns those with the best score
    maxi=0
    avg=0
    best_tab=[]
    for i in tab:
        if(Count_value(i)>maxi):
            maxi=Count_value(i)
            best_tab.append(i)
        elif(Count_value(i)==maxi):#If the max value is the same, the array is added to the max solutions.
            best_tab.append(i)
        avg=avg+Count_value(i)
    print(avg/500)
    return best_tab
    
def Count_4(tab):
    #Count the number of 4s in the grids=0
    s=0
    for i in range(len(tab)):
        if tab[i]==4:
            s=s+1
    return s

# Informed Search

We decided to collect 500 solutions before stopping the search, so that we could have several results, but also the best possible one. The number 500 was chosen to avoid the a_star running too long. 

What's more, after all the possible cookies have been added, we also decide to add a blank space to ensure that all possible grids are checked. We don't put it as a cookie because we're sure we'll be able to place an empty space.


In [8]:
def best_first_search(problem, f):
    """We implement a best first search and then use astar. With the following parameters:
        
        Parameters:
        - problem: Problem used for best first search
        - f: The heuristic used
    """
    ind = 0
    init = problem.initial
    checked=0
    frontier = []
    solutions=[]
    heapq.heappush(frontier, (0, init))
    explored = set()

    while len(solutions)<500 and frontier:#We stop the program at 500 solutions so as not to waste too much time while having a good sample to find the best possible results.
        cost, curent = heapq.heappop(frontier)
        checked=checked+1
        if problem.goal_test(curent):
            solutions.append(curent)#When the grid is complete, it is added to the solutions
        ind=get_index(curent)
        for bis in check_possible_biscuits(curent, problem.Biscuits, ind, problem.error):#We look at the cookies that can be placed in the first free position.
            child = add_biscuit(curent, bis, ind)#we create grids that add one possible cookie per grid
            heapq.heappush(frontier, (-f(child), child)) 
        if(ind<500):
            child=add_empty(curent)#We add an empty box to settle the case if no cookies can be placed and to make all possible cases.
            heapq.heappush(frontier, (-f(child), child))
    print ("Number of element in frontier : ",len(frontier))
    print ("Number of element checked : ",checked)
    return solutions
    
    
    

Run the algorithm

In [9]:
tab=[0]*500
def astar_search(problem):
    tab=best_first_search(P,P.h2)
    return(tab)

P=Probleme_biscuit(tab,Biscuits,error)
start_time = time.time()#Time calculation
b=astar_search(P)
end_time = time.time()
execution_time = end_time - start_time
tab_result=best(b)
#Performance display
print("Time : ",execution_time, "s ")
print(Count_value(tab_result[0]))
print(tab_result)

Number of element in frontier :  9037
Number of element checked :  8186
706.126
Time :  2.6669957637786865 s 
715
[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, -1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4, -1, 1, 1, 1, 1, -1, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, -1, 2, 2, 2, 2, 2, 2, 2, 2, -1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 4, 4, 4, 4, 4, -1, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

To solve this problem, we create a star algorithm with a heuristic that calculates the maximum potential cost to be obtained with the rest of the unused grid. We use a cost of 2.3 per square because, on average, a good cookie yields a value of 1.3, and when we remove an empty space from the grid, we remove a -1, which brings us to a profit of 2.3. 

In addition, we want to favour the use of cookie 4 over the use of cookie 2, so we use the Count_4 function to count the number of 4s in the grid and then minimize this value to optimize the heuristic: we find 0.1 to get the best results.

# CSP and Dynamic Programming 

## CSP Implementation using Or Tools

First we create some useful variables that we will need for our implementation.

In [5]:

# Create instances of Biscuit with their respective attributes
B1 = Biscuit(4, 3, 4, 2, 3, 0)
B2 = Biscuit(8, 12, 5, 4, 4, 1)
B3 = Biscuit(2, 1, 1, 2, 1, 2)
B4 = Biscuit(5, 8, 2, 3, 2, 3)
Biscuits = [B1, B2, B3, B4]

# Dictionary to hold variables representing biscuit placements
variable_biscuit = {}

# Determine the maximum length of biscuits and valid start positions
max_biscuit_length = max(biscuit.length for biscuit in Biscuits)
max_start_pos = len(error) - max_biscuit_length
print(max_start_pos)
model = cp_model.CpModel()
# Create variables for possible biscuit positions
for biscuit in Biscuits:
    for i in range(max_start_pos + 1):
        nom_var = f"biscuit_{biscuit.index}_pos_{i}"
        variable_biscuit[(biscuit.index, i)] = model.NewBoolVar(nom_var)
print(variable_biscuit[(1, 492)])


492
biscuit_1_pos_492


We now create the constraints needed for the csp implementation. 

In [6]:
# Constraint 1: Ensure no overlapping biscuits
for i in range(len(error)):
    overlap_biscuits = []  # List of biscuits that could overlap at position i
    for biscuit in Biscuits:
        # Determine valid initial positions for biscuits that could overlap at position i
        position_init = range(
            max(0, i - biscuit.length + 1),
            min(i + 1, max_start_pos + 1)
        )
        for pos_init in position_init:
            if pos_init <= i < pos_init + biscuit.length:
                overlap_biscuits.append(variable_biscuit[(biscuit.index, pos_init)])
    model.Add(sum(overlap_biscuits) <= 1)  # At most one biscuit can occupy a position

# Constraint 2: Ensure defect thresholds are met for placed biscuits
for biscuit in Biscuits:
    for start_pos in range(max_start_pos):
        # Calculate total defects covered by the biscuit at start_pos
        total_a = 0
        total_b = 0
        total_c = 0
        for i in range(start_pos, start_pos + biscuit.length):
            a1 = error[i].a
            b1 = error[i].b
            c1 = error[i].c
            total_a += a1
            total_b += b1
            total_c += c1

        # Add constraints to enforce defect thresholds if the biscuit is placed
        start_var = variable_biscuit[(biscuit.index, start_pos)]
        model.Add(total_a <= biscuit.a).OnlyEnforceIf(start_var)
        model.Add(total_b <= biscuit.b).OnlyEnforceIf(start_var)
        model.Add(total_c <= biscuit.c).OnlyEnforceIf(start_var)

# Create a dictionary for boolean variables representing covered positions
covered_position = {}
for x in range(500):
    covered_position[x] = model.NewBoolVar(f'position_covered_{x}')

# Calculate coverage for each position
for x in range(500):
    covered_biscuits = []  # List of biscuits that can cover position x
    for biscuit in Biscuits:
        # Determine valid start positions for biscuits to cover position x
        start_positions = range(
            max(0, x - biscuit.length + 1),
            min(x + 1, max_start_pos + 1)
        )
        for start_pos in start_positions:
            if start_pos <= x < start_pos + biscuit.length:
                covered_biscuits.append(variable_biscuit[(biscuit.index, start_pos)])
    # Add constraint to indicate if position x is covered
    model.Add(covered_position[x] == sum(covered_biscuits))

We now create the function that the model will try to maximize.

In [7]:
# Calculate the total value of placed biscuits
total_biscuit_value = sum(
    biscuit.value * variable_biscuit[(biscuit.index, start_pos)]
    for biscuit in Biscuits
    for start_pos in range(max_start_pos + 1)
)

# Calculate total number of positions covered
total_positions_covered = sum(covered_position[x] for x in range(500))

# Calculate penalty for uncovered positions
total_penalty = 500 - total_positions_covered

# Calculate the total value of the model (biscuit value minus penalty)
total_value = total_biscuit_value - total_penalty

# Maximize the total value of the model
model.Maximize(total_value)

And solve it using the Or tool solver

In [14]:
# Solve the model
start = time.time()
solver = cp_model.CpSolver()
status = solver.Solve(model)
stop = time.time() - start
print("Runtime:", stop)

Runtime: 0.30802392959594727


And then display the max value found and the best cookie arrangement:

In [15]:
#Output results
solution = []
for biscuit in Biscuits:
    for start_pos in range(max_start_pos + 1):
        if solver.Value(variable_biscuit[(biscuit.index, start_pos)]) == 1:
            # If true, the biscuit is placed at the start_pos
            solution.append((biscuit.index, start_pos))

print("Maximum Value :", solver.ObjectiveValue(),
      "Biscuits placement :") 
sorted_list = sorted(solution, key=lambda x: x[1])
print(sorted_list)

Maximum Value : 715.0 Biscuits placement :
[(1, 0), (1, 8), (2, 16), (1, 18), (3, 26), (3, 31), (0, 36), (1, 40), (2, 48), (0, 50), (3, 54), (1, 59), (0, 67), (1, 71), (3, 79), (3, 84), (1, 89), (1, 98), (2, 106), (3, 108), (0, 114), (1, 119), (3, 127), (1, 132), (0, 140), (3, 144), (1, 149), (1, 158), (1, 167), (1, 176), (2, 185), (1, 187), (3, 195), (3, 200), (3, 205), (3, 210), (2, 215), (3, 217), (3, 222), (3, 227), (3, 232), (3, 237), (1, 242), (3, 250), (3, 255), (3, 260), (3, 265), (1, 270), (1, 278), (1, 286), (1, 294), (3, 302), (3, 307), (1, 312), (3, 320), (1, 325), (3, 333), (3, 338), (3, 343), (3, 348), (0, 353), (1, 357), (0, 365), (2, 369), (0, 371), (1, 375), (3, 383), (3, 388), (3, 393), (1, 398), (1, 406), (3, 414), (3, 419), (1, 424), (1, 432), (1, 440), (3, 448), (1, 453), (3, 461), (3, 466), (1, 471), (1, 479), (3, 487), (1, 492)]


# BONUSES

We decided to go a step further and implement a Dynamic programming approach and nature inspired models. 


## Dynamic programming Implementation

We first define two useful sections for our Dynamic programming implementation: 

In [16]:
def obtain_value(roll, position):
    """
    Calculates the total value of a roll of biscuits based on their position.
    
    Args:
        roll (list): A list of tuples containing biscuits and their positions.
        position (int): The position being evaluated.

    Returns:
        int: The total value of the roll adjusted for unoccupied positions.
    """
    value = 0
    position_occupied = 0

    for biscuit, pos in roll:
        value += biscuit.value
        position_occupied += biscuit.length

    value -= (position - position_occupied)  # Penalize unoccupied positions
    return value


def be_placed(biscuit, start_pos, dough_defects):
    """
    Determines if a biscuit can be placed at a given position considering dough defects.
    
    Args:
        biscuit (Biscuit): The biscuit to place.
        start_pos (int): The starting position.
        dough_defects (list): A list of dough defect data.

    Returns:
        bool: True if the biscuit can be placed, False otherwise.
    """
    total_a = total_b = total_c = 0
    index_fin = start_pos + biscuit.length

    # Check if placement exceeds dough length
    if index_fin > 500:
        return False

    # Sum defects in the placement range
    for i in range(start_pos, start_pos + biscuit.length):
        if start_pos + biscuit.length != 500:
            total_a += dough_defects[i].a
            total_b += dough_defects[i].b
            total_c += dough_defects[i].c

    # Verify that defects are within biscuit tolerances
    return total_a <= biscuit.a and total_b <= biscuit.b and total_c <= biscuit.c

Here we define the dynamic programming function defines in itself: it iterates over the whole length of the dough and tries every biscuit at every position, keeping in memory the best disposition so far and updating it for every new node.

In [17]:
def maximise(biscuits_types, defects):
    """
    Maximizes the profit by placing biscuits optimally on the dough, considering defects.
    
    Args:
        types_biscuits (list): A list of Biscuit objects.
        defauts_pate (list): A list of dough defect data.

    Returns:
        tuple: The optimal biscuit placement and the maximum profit value.
    """
    max_a_index = [None] * 501  # Array to store best combinations at each position
    max_a_index[0] = []
    opti_value = 0

    for position in range(1, 501):
        max_value = -50  # Initialize with a very low value
        best_combi = None  # Track the best combination for this position

        for biscuit in biscuits_types:
            if position >= biscuit.length:
                last_combi = max_a_index[position - biscuit.length]
                if last_combi is not None:
                    actual_value = obtain_value(last_combi, position - biscuit.length)

                    debut_position = position - biscuit.length
                    if be_placed(biscuit, debut_position, defects):
                        actual_value += biscuit.value

                        temp_combi = last_combi + [(biscuit, debut_position)]
                        total_actual_value = obtain_value(temp_combi, position)

                        if total_actual_value > max_value:
                            best_combi = temp_combi
                            max_value = total_actual_value

        # Store the best combination for the current position
        if best_combi is None:
            max_a_index[position] = max_a_index[position - 1]
        else:
            max_a_index[position] = best_combi

        # Track the maximum value at position 500
        if position == 500:
            opti_value = max_value

    return max_a_index[500], opti_value


And we then build our dough roll, its defects and use the function on them to find optimal value and arrangement. Finally, we display the results.

In [18]:
# Load defect data
# Note: The functions `fill_defect` and `Biscuit` are assumed to be defined elsewhere.
defect = pd.read_csv("defects.csv")
defect = defect.sort_values(by='x')
defect2 = defect.reset_index(drop=True)
defect1 = [0] * 500

defects = fill_defect(defect1, defect2)  # Fill the defect array with defect data
B1 = Biscuit(4, 3, 4, 2, 3, 0)
B2 = Biscuit(8, 12, 5, 4, 4, 1)
B3 = Biscuit(2, 1, 1, 2, 1, 2)
B4 = Biscuit(5, 8, 2, 3, 2, 3)
biscuits = [B1, B2, B3, B4]
# Optimize biscuit placement
start = time.time()
placement, value = maximise(biscuits, defects)
stop = time.time() - start

# Output results
print("Runtime :",stop)
print(f"Maximum Value: {value}")

Runtime : 0.048003196716308594
Maximum Value: 715


# Nature inspired models

## Class Implementation

For this nature inspired part we will keep the same problem implementation that is the most simple way of creating biscuits and constraints. 
The roll initialization will be done for each algorithm.

The defects are loaded using a dictionary to make them accessible quickly.

In [18]:

# Define a class to represent a Biscuit
class Biscuit:
    def __init__(self, biscuit_id, length, value, max_defects):
        self.biscuit_id = biscuit_id
        self.length = length
        self.value = value
        self.max_defects = max_defects


# Load biscuits
biscuits = [
    Biscuit(0, 4, 3, {'a': 4, 'b': 2, 'c': 3}),
    Biscuit(1, 8, 12, {'a': 5, 'b': 4, 'c': 4}),
    Biscuit(2, 2, 1, {'a': 1, 'b': 2, 'c': 1}),
    Biscuit(3, 5, 8, {'a': 2, 'b': 3, 'c': 2}),
]

# Function to load defects from CSV
def load_defects(file_path):
    defects = defaultdict(lambda: defaultdict(int))
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            position = round(float(row['x']))  # Round position to the nearest integer
            defect_type = row['class']
            defects[position][defect_type] += 1
    return defects

# Load defects from the provided CSV file
defects = load_defects("defects.csv")


## Genetic Algorithm

### Simplest Approach

### First approach using simple crossover and a basic mutation. 

No particular method of population selection was used. 



In [19]:

class GeneticAlgorithm:
    def __init__(self, biscuits, roll_length, defects, population_size=500, generations=1000, mutation_rate=0.3):
        """Initialize the Genetic Algorithm with the given parameters.
        
        Parameters:
        - biscuits: List of biscuit objects, each with properties like value, length, and max_defects.
        - roll_length: Length of the roll where biscuits need to be placed.
        - defects: Dictionary representing defects in the roll.
        - population_size: Number of individuals in the population.
        - generations: Number of generations to run the algorithm.
        - mutation_rate: Probability of mutation occurring in a child solution.
        """
        self.biscuits = biscuits
        self.roll_length = roll_length
        self.defects = defects
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.population = []
    
    def generate_random_solution(self):
        """Generate a random solution (individual) by placing biscuits in the roll.
        
        This method prioritizes biscuits with IDs 2 and 4 by assigning them higher weights, as they are more likely
        to appear in high-value solutions.
        """
        solution = []
        used_positions = set()
        biscuit_weights = {2: 0.8, 4: 0.8}  # Prioritize Biscuits 2 and 4 with higher weights

        while True:
            if random.random() < 0.5:
                # Prefer Biscuits 2 and 4 using weighted selection
                chosen_id = random.choices(
                    [b.biscuit_id for b in self.biscuits],
                    weights=[
                        biscuit_weights.get(b.biscuit_id, 0.2) for b in self.biscuits
                    ]
                )[0]
                biscuit = next(b for b in self.biscuits if b.biscuit_id == chosen_id)
            else:
                # Select a random biscuit from the entire list
                biscuit = random.choice(self.biscuits)

            # Randomly choose a position for the biscuit
            position = random.randint(0, self.roll_length - biscuit.length)

            # Ensure the biscuit does not overlap with others and satisfies defect constraints
            if not any(pos in used_positions for pos in range(position, position + biscuit.length)):
                if self.is_valid_biscuit(biscuit, position):
                    solution.append((biscuit.biscuit_id, position))
                    used_positions.update(range(position, position + biscuit.length))

            # Stop condition for random generation
            if random.random() < 0.1:
                break
        return solution

    def is_valid_biscuit(self, biscuit, position):
        """Check if a biscuit can be placed at a specific position without exceeding defect constraints.
        
        Parameters:
        - biscuit: The biscuit to be placed.
        - position: Starting position for the biscuit.
        
        Returns:
        - True if the biscuit can be placed; False otherwise.
        """
        covered_defects = defaultdict(int)
        for pos in range(position, position + biscuit.length):
            for defect_type in self.defects.get(pos, {}):
                covered_defects[defect_type] += self.defects[pos][defect_type]
        
        # Ensure the defects covered by the biscuit do not exceed its allowed limits
        return all(covered_defects[defect] <= biscuit.max_defects.get(defect, 0) for defect in biscuit.max_defects)

    def fitness(self, solution):
        """Calculate the fitness of a solution.
        
        Fitness is computed as the total value of biscuits minus the penalty for unused positions.
        
        Parameters:
        - solution: A list of tuples representing placed biscuits and their positions.
        
        Returns:
        - Fitness score as an integer.
        """
        used_positions = set()
        total_value = 0

        for biscuit_id, position in solution:
            biscuit = next(b for b in self.biscuits if b.biscuit_id == biscuit_id)
            total_value += biscuit.value
            used_positions.update(range(position, position + biscuit.length))
        
        penalty = self.roll_length - len(used_positions)  # Unused positions as penalty
        return total_value - penalty

    def crossover(self, parent1, parent2):
        """Perform crossover between two parent solutions to produce a child solution.
        
        The child is a combination of parts from both parents, ensuring no duplicate biscuits.
        """
        split_point = random.randint(0, len(parent1))
        child = parent1[:split_point] + [b for b in parent2 if b not in parent1]
        return self.repair_solution(child)

    def repair_solution(self, solution):
        """Repair a solution to ensure it adheres to constraints (no overlap, no defect violations).
        
        Parameters:
        - solution: A list of tuples representing placed biscuits and their positions.
        
        Returns:
        - Repaired solution as a list of tuples.
        """
        repaired = []
        used_positions = set()
        for biscuit_id, position in solution:
            biscuit = next(b for b in self.biscuits if b.biscuit_id == biscuit_id)
            if not any(pos in used_positions for pos in range(position, position + biscuit.length)):
                if self.is_valid_biscuit(biscuit, position):
                    repaired.append((biscuit_id, position))
                    used_positions.update(range(position, position + biscuit.length))
        return repaired

    def mutate(self, solution):
        """Perform mutation by adding a random biscuit to the solution.
        
        Parameters:
        - solution: A list of tuples representing placed biscuits and their positions.
        
        Returns:
        - Mutated solution after repairing.
        """
        if random.random() < self.mutation_rate:
            biscuit = random.choice(self.biscuits)
            position = random.randint(0, self.roll_length - biscuit.length)
            if self.is_valid_biscuit(biscuit, position):
                solution.append((biscuit.biscuit_id, position))
        return self.repair_solution(solution)

    def run(self):
        """Run the Genetic Algorithm to optimize biscuit placement.
        
        The algorithm evolves the population over a defined number of generations, selecting the best solutions
        and introducing diversity through crossover and mutation.
        
        Returns:
        - Best solution found after all generations.
        """
        self.population = [self.generate_random_solution() for _ in range(self.population_size)]

        for generation in range(self.generations):
            # Evaluate fitness of all individuals in the population
            fitness_scores = [(self.fitness(individual), individual) for individual in self.population]
            fitness_scores.sort(reverse=True, key=lambda x: x[0])

            # Select the top 50% of the population to create the next generation
            next_generation = [individual for _, individual in fitness_scores[:self.population_size // 2]]

            # Create new individuals through crossover and mutation until population size is restored
            while len(next_generation) < self.population_size:
                parent1, parent2 = random.sample(next_generation, 2)
                child = self.crossover(parent1, parent2)
                child = self.mutate(child)
                next_generation.append(child)

            self.population = next_generation
            best_fitness = fitness_scores[0][0]
            print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")

        # Return the best solution found
        best_solution = max(self.population, key=self.fitness)
        return best_solution


In [20]:
# Run the Genetic Algorithm
roll_length = 500
start=time.time()
ga = GeneticAlgorithm(biscuits, roll_length, defects)
best_solution = ga.run()
stop=time.time() - start

# Print the best solution
print("Best Solution:", best_solution)
print("Runtime : ", stop)

Generation 1: Best Fitness = -205
Generation 2: Best Fitness = -94
Generation 3: Best Fitness = -28
Generation 4: Best Fitness = 77
Generation 5: Best Fitness = 154
Generation 6: Best Fitness = 174
Generation 7: Best Fitness = 198
Generation 8: Best Fitness = 198
Generation 9: Best Fitness = 198
Generation 10: Best Fitness = 216
Generation 11: Best Fitness = 245
Generation 12: Best Fitness = 245
Generation 13: Best Fitness = 245
Generation 14: Best Fitness = 250
Generation 15: Best Fitness = 255
Generation 16: Best Fitness = 255
Generation 17: Best Fitness = 259
Generation 18: Best Fitness = 287
Generation 19: Best Fitness = 287
Generation 20: Best Fitness = 287
Generation 21: Best Fitness = 287
Generation 22: Best Fitness = 287
Generation 23: Best Fitness = 287
Generation 24: Best Fitness = 302
Generation 25: Best Fitness = 302
Generation 26: Best Fitness = 302
Generation 27: Best Fitness = 302
Generation 28: Best Fitness = 329
Generation 29: Best Fitness = 334
Generation 30: Best Fit

Generation 240: Best Fitness = 415
Generation 241: Best Fitness = 415
Generation 242: Best Fitness = 418
Generation 243: Best Fitness = 418
Generation 244: Best Fitness = 418
Generation 245: Best Fitness = 420
Generation 246: Best Fitness = 420
Generation 247: Best Fitness = 420
Generation 248: Best Fitness = 420
Generation 249: Best Fitness = 420
Generation 250: Best Fitness = 420
Generation 251: Best Fitness = 420
Generation 252: Best Fitness = 420
Generation 253: Best Fitness = 420
Generation 254: Best Fitness = 420
Generation 255: Best Fitness = 420
Generation 256: Best Fitness = 420
Generation 257: Best Fitness = 422
Generation 258: Best Fitness = 422
Generation 259: Best Fitness = 422
Generation 260: Best Fitness = 422
Generation 261: Best Fitness = 422
Generation 262: Best Fitness = 422
Generation 263: Best Fitness = 422
Generation 264: Best Fitness = 422
Generation 265: Best Fitness = 422
Generation 266: Best Fitness = 422
Generation 267: Best Fitness = 423
Generation 268: Best

Generation 476: Best Fitness = 442
Generation 477: Best Fitness = 442
Generation 478: Best Fitness = 442
Generation 479: Best Fitness = 442
Generation 480: Best Fitness = 442
Generation 481: Best Fitness = 442
Generation 482: Best Fitness = 442
Generation 483: Best Fitness = 442
Generation 484: Best Fitness = 442
Generation 485: Best Fitness = 442
Generation 486: Best Fitness = 442
Generation 487: Best Fitness = 442
Generation 488: Best Fitness = 442
Generation 489: Best Fitness = 442
Generation 490: Best Fitness = 442
Generation 491: Best Fitness = 442
Generation 492: Best Fitness = 445
Generation 493: Best Fitness = 445
Generation 494: Best Fitness = 445
Generation 495: Best Fitness = 445
Generation 496: Best Fitness = 445
Generation 497: Best Fitness = 445
Generation 498: Best Fitness = 445
Generation 499: Best Fitness = 445
Generation 500: Best Fitness = 445
Generation 501: Best Fitness = 445
Generation 502: Best Fitness = 445
Generation 503: Best Fitness = 445
Generation 504: Best

Generation 711: Best Fitness = 454
Generation 712: Best Fitness = 454
Generation 713: Best Fitness = 454
Generation 714: Best Fitness = 454
Generation 715: Best Fitness = 454
Generation 716: Best Fitness = 454
Generation 717: Best Fitness = 454
Generation 718: Best Fitness = 454
Generation 719: Best Fitness = 454
Generation 720: Best Fitness = 454
Generation 721: Best Fitness = 454
Generation 722: Best Fitness = 454
Generation 723: Best Fitness = 454
Generation 724: Best Fitness = 454
Generation 725: Best Fitness = 454
Generation 726: Best Fitness = 454
Generation 727: Best Fitness = 455
Generation 728: Best Fitness = 455
Generation 729: Best Fitness = 455
Generation 730: Best Fitness = 455
Generation 731: Best Fitness = 455
Generation 732: Best Fitness = 455
Generation 733: Best Fitness = 455
Generation 734: Best Fitness = 455
Generation 735: Best Fitness = 455
Generation 736: Best Fitness = 455
Generation 737: Best Fitness = 455
Generation 738: Best Fitness = 455
Generation 739: Best

Generation 946: Best Fitness = 475
Generation 947: Best Fitness = 475
Generation 948: Best Fitness = 475
Generation 949: Best Fitness = 475
Generation 950: Best Fitness = 475
Generation 951: Best Fitness = 475
Generation 952: Best Fitness = 475
Generation 953: Best Fitness = 475
Generation 954: Best Fitness = 479
Generation 955: Best Fitness = 479
Generation 956: Best Fitness = 479
Generation 957: Best Fitness = 479
Generation 958: Best Fitness = 479
Generation 959: Best Fitness = 479
Generation 960: Best Fitness = 479
Generation 961: Best Fitness = 479
Generation 962: Best Fitness = 479
Generation 963: Best Fitness = 479
Generation 964: Best Fitness = 479
Generation 965: Best Fitness = 479
Generation 966: Best Fitness = 479
Generation 967: Best Fitness = 479
Generation 968: Best Fitness = 479
Generation 969: Best Fitness = 479
Generation 970: Best Fitness = 479
Generation 971: Best Fitness = 479
Generation 972: Best Fitness = 479
Generation 973: Best Fitness = 479
Generation 974: Best

#### Choice of Implementation:

Generate Random solution :

We decided to do a random generate solution function that will advantage biscuit 2 and 4. We found using previous techniques(CSP,A* and backtracking) that the solutions achieving a 715 score had a lot a biscuits 2 and 4. 

Repair mechanism : 

This function was mandatory to be able to maintain an admissible solution through the generations. It removes the biscuits potentially overlapping and the biscuits how are wrongly placed due to the defects constraints. 

### Sub segment Approach

Introducing the sub segment approach :


The subsegment_crossover approach enhances solution diversity while maintaining structural integrity. It leverages divisors of the roll length to divide the problem space into smaller subsegments, allowing localized crossover between two parent solutions. 

Within each subsegment, biscuits from both parents are collected, shuffled, and filtered to retain the most valuable placements without overlapping. By limiting the number of selected biscuits per subsegment and repairing the final solution, the method ensures feasibility and fosters a balance between exploration (randomized selection) and exploitation (choosing the best configurations). 

This localized approach promotes better convergence by combining high-value elements from both parents in a controlled, modular fashion.

In [21]:
class GeneticAlgorithm:
    def __init__(self, biscuits, roll_length, defects, population_size=1000, generations=100, mutation_rate=0.4, elitism_rate=0.40):
        """
        Initializes the GeneticAlgorithm instance with input data and parameters.

        Parameters:
        - biscuits: List of available biscuits with attributes like length, value, and max_defects.
        - roll_length: Length of the roll to be optimized.
        - defects: Dictionary of defect information for each position in the roll.
        - population_size: Number of solutions in the population (default=1000).
        - generations: Number of generations for evolution (default=50).
        - mutation_rate: Probability of mutation occurring in a solution (default=0.4).
        - elitism_rate: Fraction of top solutions preserved in each generation (default=0.4).
        """
        self.biscuits = biscuits
        self.roll_length = roll_length
        self.defects = defects
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.elitism_rate = elitism_rate
        self.population = []

    def generate_random_solution(self):
        """
        Generates a random solution by placing biscuits at valid positions in the roll.

        Returns:
        - solution: List of tuples representing biscuits placed (biscuit_id, position).
        """
        solution = []
        used_positions = set()

        while True:
            biscuit = random.choice(self.biscuits)
            position = random.randint(0, self.roll_length - biscuit.length)

            # Ensure no overlap and validity of placement
            if not any(pos in used_positions for pos in range(position, position + biscuit.length)):
                if self.is_valid_biscuit(biscuit, position):
                    solution.append((biscuit.biscuit_id, position))
                    used_positions.update(range(position, position + biscuit.length))

            # Stop generation with a random condition
            if random.random() < 0.1:
                break

        return solution

    def is_valid_biscuit(self, biscuit, position):
        """
        Checks if placing a biscuit at a specific position satisfies defect constraints.

        Parameters:
        - biscuit: Biscuit object being placed.
        - position: Starting position for the biscuit.

        Returns:
        - Boolean indicating validity of the placement.
        """
        covered_defects = defaultdict(int)

        for pos in range(position, position + biscuit.length):
            for defect_type in self.defects.get(pos, {}):
                covered_defects[defect_type] += self.defects[pos][defect_type]

        return all(covered_defects[defect] <= biscuit.max_defects.get(defect, 0) for defect in biscuit.max_defects)

    def fitness(self, solution):
        """
        Evaluates the fitness of a solution based on biscuit value and unused positions.

        Parameters:
        - solution: List of tuples representing placed biscuits (biscuit_id, position).

        Returns:
        - Fitness score (higher is better).
        """
        used_positions = np.zeros(self.roll_length, dtype=bool)
        total_value = 0

        for biscuit_id, position in solution:
            biscuit = next(b for b in self.biscuits if b.biscuit_id == biscuit_id)
            total_value += biscuit.value
            used_positions[position:position + biscuit.length] = True

        penalty = self.roll_length - np.sum(used_positions)
        return total_value - penalty

    def find_divisors(self, n):
        """
        Finds all divisors of a given number.

        Parameters:
        - n: Integer to find divisors of.

        Returns:
        - List of divisors.
        """
        return [i for i in range(1, n + 1) if n % i == 0]

    def subsegment_crossover(self, parent1, parent2):
        """
        Performs crossover between two parent solutions using subsegment logic.

        Parameters:
        - parent1: First parent solution.
        - parent2: Second parent solution.

        Returns:
        - child: New solution generated from the parents.
        """
        divisors = self.find_divisors(self.roll_length)
        divisor = random.choice(divisors) if divisors else 1
        subsegment_size = self.roll_length // divisor

        child_segments = []
        used_positions = set()

        for i in range(divisor):
            start = i * subsegment_size
            end = (i + 1) * subsegment_size

            parent1_biscuits = [b for b in parent1 if start <= b[1] < end]
            parent2_biscuits = [b for b in parent2 if start <= b[1] < end]

            all_biscuits = parent1_biscuits + parent2_biscuits
            random.shuffle(all_biscuits)

            unique_biscuits = list(set(all_biscuits))
            unique_biscuits.sort(key=lambda x: next(b.value for b in self.biscuits if b.biscuit_id == x[0]), reverse=True)

            chosen_biscuits = []
            for biscuit_id, position in unique_biscuits:
                biscuit = next(b for b in self.biscuits if b.biscuit_id == biscuit_id)
                if not any(pos in used_positions for pos in range(position, position + biscuit.length)):
                    if start <= position < end:
                        chosen_biscuits.append((biscuit_id, position))
                        used_positions.update(range(position, position + biscuit.length))
                        if len(chosen_biscuits) == 2:
                            break

            child_segments.extend(chosen_biscuits)

        return self.repair_solution(child_segments)

    def repair_solution(self, solution):
        """
        Repairs a solution to ensure validity (no overlap or defects constraint violations).

        Parameters:
        - solution: Solution to be repaired.

        Returns:
        - repaired: Valid solution.
        """
        repaired = []
        used_positions = set()

        for biscuit_id, position in solution:
            biscuit = next(b for b in self.biscuits if b.biscuit_id == biscuit_id)
            if not any(pos in used_positions for pos in range(position, position + biscuit.length)):
                if self.is_valid_biscuit(biscuit, position):
                    repaired.append((biscuit_id, position))
                    used_positions.update(range(position, position + biscuit.length))

        return repaired

    def mutate(self, solution):
        """
        Mutates a solution by randomly replacing a biscuit.

        Parameters:
        - solution: Solution to mutate.

        Returns:
        - Mutated and repaired solution.
        """
        if random.random() < self.mutation_rate:
            biscuit = random.choice(self.biscuits)
            position = random.randint(0, self.roll_length - biscuit.length)
            if self.is_valid_biscuit(biscuit, position):
                solution.append((biscuit.biscuit_id, position))

        return self.repair_solution(solution)

    def run(self):
        """
        Executes the genetic algorithm, evolving the population over multiple generations.

        Returns:
        - best_solution: The best solution found.
        """
        self.population = [self.generate_random_solution() for _ in range(self.population_size)]
        elitism_count = int(self.population_size * self.elitism_rate)

        for generation in range(self.generations):
            fitness_scores = [(self.fitness(individual), individual) for individual in self.population]
            fitness_scores.sort(reverse=True, key=lambda x: x[0])

            next_generation = [individual for _, individual in fitness_scores[:elitism_count]]

            while len(next_generation) < self.population_size:
                parent1, parent2 = random.sample(next_generation, 2)
                child = self.subsegment_crossover(parent1, parent2)
                child = self.mutate(child)
                next_generation.append(child)

            self.population = next_generation
            best_fitness = fitness_scores[0][0]
            print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")

        return max(self.population, key=self.fitness)


#### Choice of Implementation:

Generate Random solution: 

We generate completely random solution as we want to keep exploration high. The sub segment and elitism approach offer a high level of exploitation and rapid convergence. 

Repair mechanism : 

This function was mandatory to be able to maintain an admissible solution through the generations. It removes the biscuits potentially overlapping and the biscuits how are wrongly placed due to the defects constraints. 

Elitism:

Elitism is a technique in genetic algorithms where a certain number of the best-performing solutions (individuals) from the current generation are directly carried over to the next generation without modification.

In [22]:
# Run the Genetic Algorithm
roll_length = 500
start=time.time()
ga = GeneticAlgorithm(biscuits, roll_length, defects)
best_solution = ga.run()
stop=time.time() - start

# Print the best solution
print("Best Solution:", best_solution)
print("Runtime: ",stop)

Generation 1: Best Fitness = -180
Generation 2: Best Fitness = 62
Generation 3: Best Fitness = 318
Generation 4: Best Fitness = 390
Generation 5: Best Fitness = 454
Generation 6: Best Fitness = 489
Generation 7: Best Fitness = 542
Generation 8: Best Fitness = 557
Generation 9: Best Fitness = 564
Generation 10: Best Fitness = 588
Generation 11: Best Fitness = 589
Generation 12: Best Fitness = 592
Generation 13: Best Fitness = 592
Generation 14: Best Fitness = 592
Generation 15: Best Fitness = 608
Generation 16: Best Fitness = 608
Generation 17: Best Fitness = 608
Generation 18: Best Fitness = 608
Generation 19: Best Fitness = 611
Generation 20: Best Fitness = 614
Generation 21: Best Fitness = 614
Generation 22: Best Fitness = 619
Generation 23: Best Fitness = 622
Generation 24: Best Fitness = 625
Generation 25: Best Fitness = 627
Generation 26: Best Fitness = 628
Generation 27: Best Fitness = 631
Generation 28: Best Fitness = 632
Generation 29: Best Fitness = 632
Generation 30: Best Fit

### Sub segment and progressive approach

In order to achieve better scores we decided to sub divide the roll. The algorithm will learn on smaller sized rolls and progressively learn on bigger ones. 
The overall problem for the genetic approach is the diversity. 

In [23]:
class GeneticAlgorithm:
    def __init__(self, biscuits, initial_roll_length, defects, population_size=1000, generations=50, mutation_rate=0.4, elitism_rate=0.40, max_roll_length=500):
        """
        Initialize the Genetic Algorithm with required parameters.

        Parameters:
        - biscuits: List of available biscuits.
        - initial_roll_length: Initial roll length to start optimization.
        - defects: Dictionary representing defect positions and counts.
        - population_size: Number of solutions in each generation.
        - generations: Number of generations for evolution.
        - mutation_rate: Probability of mutation for a solution.
        - elitism_rate: Proportion of top solutions to retain in each generation.
        - max_roll_length: Maximum roll length to consider during gradual training.
        """
        self.biscuits = biscuits
        self.roll_length = initial_roll_length
        self.defects = defects
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.elitism_rate = elitism_rate
        self.max_roll_length = max_roll_length
        self.population = []  # Stores the population of solutions

    def generate_random_solution(self):
        """
        Generate a random solution by placing biscuits at valid positions on the roll.
        
        Returns:
        - A list of tuples representing biscuit placements (biscuit_id, position).
        """
        solution = []
        used_positions = set()

        while True:
            biscuit = random.choice(self.biscuits)
            position = random.randint(0, self.roll_length - biscuit.length)

            # Check if the biscuit can be placed without overlap
            if not any(pos in used_positions for pos in range(position, position + biscuit.length)):
                if self.is_valid_biscuit(biscuit, position):
                    solution.append((biscuit.biscuit_id, position))
                    used_positions.update(range(position, position + biscuit.length))

            # Random stop condition for generating a solution
            if random.random() < 0.1:
                break

        return solution

    def is_valid_biscuit(self, biscuit, position):
        """
        Check if a biscuit can be placed at a specific position on the roll without violating defect constraints.

        Parameters:
        - biscuit: Biscuit to be placed.
        - position: Starting position on the roll.

        Returns:
        - True if the biscuit placement is valid, otherwise False.
        """
        covered_defects = defaultdict(int)

        # Calculate the defects covered by this biscuit
        for pos in range(position, position + biscuit.length):
            for defect_type in self.defects.get(pos, {}):
                covered_defects[defect_type] += self.defects[pos][defect_type]

        # Ensure defect constraints are not violated
        return all(covered_defects[defect] <= biscuit.max_defects.get(defect, 0) for defect in biscuit.max_defects)

    def fitness(self, solution):
        """
        Calculate the fitness of a solution based on biscuit values and penalties for unused roll positions.

        Parameters:
        - solution: List of biscuit placements (biscuit_id, position).

        Returns:
        - Fitness value as a numerical score.
        """
        used_positions = np.zeros(self.roll_length, dtype=bool)
        total_value = 0

        for biscuit_id, position in solution:
            biscuit = next(b for b in self.biscuits if b.biscuit_id == biscuit_id)
            total_value += biscuit.value
            used_positions[position:position + biscuit.length] = True

        # Penalize unused positions
        penalty = self.roll_length - np.sum(used_positions)
        return total_value - penalty

    def find_divisors(self, n):
        """
        Find all divisors of a given number.

        Parameters:
        - n: Integer value.

        Returns:
        - List of divisors of n.
        """
        divisors = []
        for i in range(1, n + 1):
            if n % i == 0:
                divisors.append(i)
        return divisors

    def subsegment_crossover(self, parent1, parent2):
        """
        Perform crossover between two parents by combining their best subsegments.

        Parameters:
        - parent1: First parent solution.
        - parent2: Second parent solution.

        Returns:
        - Child solution after crossover and repair.
        """
        divisors = self.find_divisors(self.roll_length)

        if divisors:
            # Choose a random divisor and calculate subsegment size
            divisor = random.choice(divisors)
            subsegment_size = self.roll_length // divisor

        child_segments = []
        used_positions = set()

        for i in range(divisor):
            start = i * subsegment_size
            end = (i + 1) * subsegment_size

            # Collect biscuits from both parents within the subsegment
            parent1_biscuits = [b for b in parent1 if start <= b[1] < end]
            parent2_biscuits = [b for b in parent2 if start <= b[1] < end]

            # Shuffle and select unique biscuits by value
            all_biscuits = parent1_biscuits + parent2_biscuits
            random.shuffle(all_biscuits)
            unique_biscuits = list(set(all_biscuits))
            unique_biscuits.sort(key=lambda x: next(b.value for b in self.biscuits if b.biscuit_id == x[0]), reverse=True)

            # Select up to 2 biscuits per subsegment
            chosen_biscuits = []
            for biscuit_id, position in unique_biscuits:
                biscuit = next(b for b in self.biscuits if b.biscuit_id == biscuit_id)
                if not any(pos in used_positions for pos in range(position, position + biscuit.length)):
                    if start <= position < end:
                        chosen_biscuits.append((biscuit_id, position))
                        used_positions.update(range(position, position + biscuit.length))
                        if len(chosen_biscuits) == 2:
                            break

            child_segments.extend(chosen_biscuits)

        return self.repair_solution(child_segments)

    def repair_solution(self, solution):
        """
        Repair a solution to ensure it satisfies overlap and defect constraints.

        Parameters:
        - solution: List of biscuit placements (biscuit_id, position).

        Returns:
        - Repaired solution.
        """
        repaired = []
        used_positions = set()

        for biscuit_id, position in solution:
            biscuit = next(b for b in self.biscuits if b.biscuit_id == biscuit_id)
            if not any(pos in used_positions for pos in range(position, position + biscuit.length)):
                if self.is_valid_biscuit(biscuit, position):
                    repaired.append((biscuit_id, position))
                    used_positions.update(range(position, position + biscuit.length))

        return repaired

    def mutate(self, solution):
        """
        Mutate a solution by randomly adding or replacing a biscuit.

        Parameters:
        - solution: List of biscuit placements (biscuit_id, position).

        Returns:
        - Mutated and repaired solution.
        """
        if random.random() < self.mutation_rate:
            biscuit = random.choice(self.biscuits)
            position = random.randint(0, self.roll_length - biscuit.length)
            if self.is_valid_biscuit(biscuit, position):
                solution.append((biscuit.biscuit_id, position))

        return self.repair_solution(solution)

    def run(self):
        """
        Execute the genetic algorithm, gradually increasing the roll length for training.

        Returns:
        - Best solution found after optimization.
        """
        current_roll_length = self.roll_length
        best_solution = None

        while current_roll_length <= self.max_roll_length:
            print(f"Training for roll length: {current_roll_length}")

            self.roll_length = current_roll_length

            # Initialize population, starting with the previous best solution if available
            if best_solution:
                self.population = [best_solution] + [self.generate_random_solution() for _ in range(self.population_size - 1)]
            else:
                self.population = [self.generate_random_solution() for _ in range(self.population_size)]

            elitism_count = int(self.population_size * self.elitism_rate)

            for generation in range(self.generations):
                fitness_scores = [(self.fitness(individual), individual) for individual in self.population]
                fitness_scores.sort(reverse=True, key=lambda x: x[0])

                # Preserve elite solutions
                next_generation = [individual for _, individual in fitness_scores[:elitism_count]]

                # Generate children through crossover and mutation
                while len(next_generation) < self.population_size:
                    parent1, parent2 = random.sample(next_generation, 2)
                    child = self.subsegment_crossover(parent1, parent2)
                    child = self.mutate(child)
                    next_generation.append(child)

                self.population = next_generation

                best_fitness = fitness_scores[0][0]
                print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")

            best_solution = max(self.population, key=self.fitness)
            current_roll_length += 20  # Gradually increase roll size

        return best_solution


#### Choice of implementation:

Progressive learning:

We decided to add a progressive learning approach to make sure the algorithm is able to learn the best solutions through time. 

In [24]:
# Run the Genetic Algorithm
roll_length = 500
# Initialize and run the genetic algorithm
start=time.time()
ga = GeneticAlgorithm(biscuits, initial_roll_length=20, defects=defects)
best_solution = ga.run()
stop=time.time() - start
print("Best Solution:", best_solution)
print("Runtime: ",stop)

Training for roll length: 20
Generation 1: Best Fitness = 27
Generation 2: Best Fitness = 27
Generation 3: Best Fitness = 27
Generation 4: Best Fitness = 27
Generation 5: Best Fitness = 27
Generation 6: Best Fitness = 27
Generation 7: Best Fitness = 27
Generation 8: Best Fitness = 27
Generation 9: Best Fitness = 27
Generation 10: Best Fitness = 27
Generation 11: Best Fitness = 27
Generation 12: Best Fitness = 27
Generation 13: Best Fitness = 27
Generation 14: Best Fitness = 27
Generation 15: Best Fitness = 27
Generation 16: Best Fitness = 27
Generation 17: Best Fitness = 27
Generation 18: Best Fitness = 27
Generation 19: Best Fitness = 27
Generation 20: Best Fitness = 27
Generation 21: Best Fitness = 27
Generation 22: Best Fitness = 27
Generation 23: Best Fitness = 27
Generation 24: Best Fitness = 27
Generation 25: Best Fitness = 27
Generation 26: Best Fitness = 27
Generation 27: Best Fitness = 27
Generation 28: Best Fitness = 27
Generation 29: Best Fitness = 27
Generation 30: Best Fit

Generation 44: Best Fitness = 129
Generation 45: Best Fitness = 129
Generation 46: Best Fitness = 129
Generation 47: Best Fitness = 129
Generation 48: Best Fitness = 129
Generation 49: Best Fitness = 129
Generation 50: Best Fitness = 129
Training for roll length: 120
Generation 1: Best Fitness = 109
Generation 2: Best Fitness = 129
Generation 3: Best Fitness = 139
Generation 4: Best Fitness = 149
Generation 5: Best Fitness = 149
Generation 6: Best Fitness = 149
Generation 7: Best Fitness = 149
Generation 8: Best Fitness = 149
Generation 9: Best Fitness = 149
Generation 10: Best Fitness = 150
Generation 11: Best Fitness = 150
Generation 12: Best Fitness = 150
Generation 13: Best Fitness = 150
Generation 14: Best Fitness = 150
Generation 15: Best Fitness = 150
Generation 16: Best Fitness = 150
Generation 17: Best Fitness = 150
Generation 18: Best Fitness = 150
Generation 19: Best Fitness = 150
Generation 20: Best Fitness = 150
Generation 21: Best Fitness = 150
Generation 22: Best Fitness

Generation 32: Best Fitness = 242
Generation 33: Best Fitness = 242
Generation 34: Best Fitness = 242
Generation 35: Best Fitness = 242
Generation 36: Best Fitness = 242
Generation 37: Best Fitness = 242
Generation 38: Best Fitness = 242
Generation 39: Best Fitness = 242
Generation 40: Best Fitness = 242
Generation 41: Best Fitness = 242
Generation 42: Best Fitness = 242
Generation 43: Best Fitness = 242
Generation 44: Best Fitness = 242
Generation 45: Best Fitness = 242
Generation 46: Best Fitness = 242
Generation 47: Best Fitness = 242
Generation 48: Best Fitness = 242
Generation 49: Best Fitness = 242
Generation 50: Best Fitness = 242
Training for roll length: 220
Generation 1: Best Fitness = 222
Generation 2: Best Fitness = 222
Generation 3: Best Fitness = 258
Generation 4: Best Fitness = 258
Generation 5: Best Fitness = 258
Generation 6: Best Fitness = 258
Generation 7: Best Fitness = 258
Generation 8: Best Fitness = 259
Generation 9: Best Fitness = 266
Generation 10: Best Fitness

Generation 20: Best Fitness = 387
Generation 21: Best Fitness = 387
Generation 22: Best Fitness = 387
Generation 23: Best Fitness = 387
Generation 24: Best Fitness = 387
Generation 25: Best Fitness = 391
Generation 26: Best Fitness = 391
Generation 27: Best Fitness = 391
Generation 28: Best Fitness = 391
Generation 29: Best Fitness = 391
Generation 30: Best Fitness = 391
Generation 31: Best Fitness = 391
Generation 32: Best Fitness = 391
Generation 33: Best Fitness = 391
Generation 34: Best Fitness = 391
Generation 35: Best Fitness = 391
Generation 36: Best Fitness = 391
Generation 37: Best Fitness = 391
Generation 38: Best Fitness = 391
Generation 39: Best Fitness = 391
Generation 40: Best Fitness = 391
Generation 41: Best Fitness = 391
Generation 42: Best Fitness = 391
Generation 43: Best Fitness = 391
Generation 44: Best Fitness = 391
Generation 45: Best Fitness = 391
Generation 46: Best Fitness = 391
Generation 47: Best Fitness = 391
Generation 48: Best Fitness = 391
Generation 49:

Generation 8: Best Fitness = 516
Generation 9: Best Fitness = 516
Generation 10: Best Fitness = 516
Generation 11: Best Fitness = 516
Generation 12: Best Fitness = 516
Generation 13: Best Fitness = 516
Generation 14: Best Fitness = 523
Generation 15: Best Fitness = 523
Generation 16: Best Fitness = 523
Generation 17: Best Fitness = 523
Generation 18: Best Fitness = 523
Generation 19: Best Fitness = 523
Generation 20: Best Fitness = 523
Generation 21: Best Fitness = 523
Generation 22: Best Fitness = 523
Generation 23: Best Fitness = 523
Generation 24: Best Fitness = 523
Generation 25: Best Fitness = 523
Generation 26: Best Fitness = 523
Generation 27: Best Fitness = 523
Generation 28: Best Fitness = 523
Generation 29: Best Fitness = 523
Generation 30: Best Fitness = 523
Generation 31: Best Fitness = 523
Generation 32: Best Fitness = 523
Generation 33: Best Fitness = 523
Generation 34: Best Fitness = 523
Generation 35: Best Fitness = 523
Generation 36: Best Fitness = 523
Generation 37: B

Generation 47: Best Fitness = 639
Generation 48: Best Fitness = 639
Generation 49: Best Fitness = 639
Generation 50: Best Fitness = 639
Training for roll length: 500
Generation 1: Best Fitness = 619
Generation 2: Best Fitness = 619
Generation 3: Best Fitness = 619
Generation 4: Best Fitness = 636
Generation 5: Best Fitness = 636
Generation 6: Best Fitness = 652
Generation 7: Best Fitness = 652
Generation 8: Best Fitness = 652
Generation 9: Best Fitness = 655
Generation 10: Best Fitness = 662
Generation 11: Best Fitness = 662
Generation 12: Best Fitness = 662
Generation 13: Best Fitness = 662
Generation 14: Best Fitness = 672
Generation 15: Best Fitness = 672
Generation 16: Best Fitness = 672
Generation 17: Best Fitness = 675
Generation 18: Best Fitness = 675
Generation 19: Best Fitness = 675
Generation 20: Best Fitness = 675
Generation 21: Best Fitness = 675
Generation 22: Best Fitness = 675
Generation 23: Best Fitness = 675
Generation 24: Best Fitness = 675
Generation 25: Best Fitness

## Ant Colony Optimization

This approach employs an Ant Colony Optimization (ACO) algorithm to optimize biscuit placement on a roll, considering constraints such as defect coverage and biscuit value. 

Each "ant" constructs a solution by iteratively choosing biscuits or skipping positions based on a probability influenced by pheromone levels (indicating historical success) and heuristic information (biscuit value). After all ants complete their solutions, the pheromone levels are updated to reinforce good solutions and decay less effective ones, guiding future iterations. 

Additionally, a grid search is used to fine-tune the parameters of the algorithm, such as pheromone influence (alpha), heuristic importance (beta), and evaporation rate, ensuring robust performance. This adaptive method balances exploration and exploitation, converging toward an optimal or near-optimal arrangement over multiple iterations.

In [25]:

class ACO:
    """
    Ant Colony Optimization (ACO) algorithm for solving the biscuit placement problem on a roll,
    considering defect constraints and maximizing value.

    Attributes:
        roll_length (int): Length of the roll.
        biscuits (list): List of Biscuit objects representing available biscuits.
        defects (dict): Mapping of roll positions to defect types and counts.
        num_ants (int): Number of ants used per iteration.
        num_iterations (int): Number of iterations to perform.
        alpha (float): Weight of pheromone influence in the probability calculation.
        beta (float): Weight of heuristic influence in the probability calculation.
        evaporation_rate (float): Rate at which pheromones decay after each iteration.
        pheromones (list): Pheromone levels for each biscuit and the skip option.
    """

    def __init__(self, roll_length, biscuits, defects, num_ants, num_iterations, alpha=1, beta=3, evaporation_rate=0.6):
        """
        Initializes the ACO algorithm with given parameters.

        Args:
            roll_length (int): Length of the roll.
            biscuits (list): List of Biscuit objects.
            defects (dict): Mapping of roll positions to defect types and counts.
            num_ants (int): Number of ants used per iteration.
            num_iterations (int): Number of iterations to perform.
            alpha (float): Weight of pheromone influence (default: 1).
            beta (float): Weight of heuristic influence (default: 3).
            evaporation_rate (float): Rate of pheromone decay (default: 0.6).
        """
        self.roll_length = roll_length
        self.biscuits = biscuits
        self.defects = defects
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.pheromones = [1] * (len(biscuits) + 1)  # Initialize pheromones for all biscuits and skip option

    def evaluate_solution(self, solution):
        """
        Evaluates the quality of a solution based on the total value and defect constraints.

        Args:
            solution (list): Sequence of biscuit IDs representing the solution.

        Returns:
            int: Total value of the solution after penalties.
        """
        total_value = 0
        current_position = 0

        for biscuit_id in solution:
            if biscuit_id == 0:  # Skip position
                total_value -= 1  # Penalize for skipping
                current_position += 1
                continue

            biscuit = self.biscuits[biscuit_id - 1]  # Adjust for 0-based indexing

            if current_position + biscuit.length > self.roll_length:
                break  # Biscuit doesn't fit, end evaluation

            # Count defects in the segment covered by this biscuit
            defects_in_segment = defaultdict(int)
            for pos in range(current_position, current_position + biscuit.length):
                for defect_type, count in self.defects[pos].items():
                    defects_in_segment[defect_type] += count

            # Check if biscuit can cover defects without exceeding its max tolerance
            if all(defects_in_segment[d] <= biscuit.max_defects.get(d, 0) for d in defects_in_segment):
                total_value += biscuit.value
                current_position += biscuit.length
            else:
                total_value -= 1  # Penalize if the biscuit cannot cover the defects

        return total_value

    def construct_solution(self):
        """
        Constructs a solution using probabilistic selection of biscuits based on pheromones and heuristics.

        Returns:
            list: Constructed solution as a sequence of biscuit IDs.
        """
        solution = []
        current_position = 0

        while current_position < self.roll_length:
            probabilities = []

            # Calculate selection probabilities for each biscuit
            for biscuit_id, biscuit in enumerate(self.biscuits):
                if current_position + biscuit.length > self.roll_length:
                    probabilities.append(0)  # Biscuit cannot fit
                    continue

                # Count defects in the segment covered by this biscuit
                defects_in_segment = defaultdict(int)
                for pos in range(current_position, current_position + biscuit.length):
                    for defect_type, count in self.defects[pos].items():
                        defects_in_segment[defect_type] += count

                # Determine if the biscuit is valid and calculate its probability
                if all(defects_in_segment[d] <= biscuit.max_defects.get(d, 0) for d in defects_in_segment):
                    heuristic = biscuit.value
                    pheromone = self.pheromones[biscuit_id]
                    probabilities.append((pheromone ** self.alpha) * (heuristic ** self.beta))
                else:
                    probabilities.append(0)  # Invalid biscuit

            # Add skipping as an option
            skip_pheromone = self.pheromones[len(self.biscuits)]
            probabilities.append((skip_pheromone ** self.alpha) * (1 ** self.beta))

            total_prob = sum(probabilities)
            if total_prob == 0:
                break  # No valid options, terminate

            probabilities = [p / total_prob for p in probabilities]
            selected_option = random.choices(range(len(self.biscuits) + 1), probabilities)[0]

            if selected_option == len(self.biscuits):  # Skip option
                solution.append(0)
                current_position += 1
            else:
                solution.append(selected_option + 1)  # Biscuit ID offset by 1
                current_position += self.biscuits[selected_option].length

        return solution

    def update_pheromones(self, solutions):
        """
        Updates pheromone levels based on the quality of solutions.

        Args:
            solutions (list): List of tuples (solution, value) from the current iteration.
        """
        # Evaporate pheromones
        self.pheromones = [p * (1 - self.evaporation_rate) for p in self.pheromones]

        # Reinforce pheromones based on solution values
        for solution, value in solutions:
            for biscuit_id in solution:
                if biscuit_id == 0:
                    self.pheromones[len(self.biscuits)] += value  # Update skip option pheromone
                else:
                    self.pheromones[biscuit_id - 1] += value

    def optimize(self):
        """
        Performs the optimization process over multiple iterations.

        Returns:
            tuple: Best solution and its value.
        """
        best_solution = None
        best_value = float('-inf')

        for _ in range(self.num_iterations):
            solutions = []

            # Generate solutions using all ants
            for _ in range(self.num_ants):
                solution = self.construct_solution()
                value = self.evaluate_solution(solution)
                solutions.append((solution, value))

                # Track the best solution
                if value > best_value:
                    best_solution = solution
                    best_value = value

            # Update pheromones based on solutions
            self.update_pheromones(solutions)

        return best_solution, best_value
# Grid Search
LENGTH = 500
NUM_ANTS = 25
NUM_ITERATIONS = 10

alpha_values = [0.5, 1, 2]
beta_values = [2, 3, 5]
evaporation_rates = [0.3, 0.6, 0.9]

best_params = None
best_overall_value = float('-inf')

for alpha in alpha_values:
    for beta in beta_values:
        for evaporation_rate in evaporation_rates:
            scores = []
            for _ in range(10):  # Run 10 times for each parameter set
                aco = ACO(LENGTH, biscuits, defects, NUM_ANTS, NUM_ITERATIONS, alpha, beta, evaporation_rate)
                _, best_value = aco.optimize()
                scores.append(best_value)

            avg_score = sum(scores) / len(scores)  # Average score over 10 runs

            print(f"Alpha: {alpha}, Beta: {beta}, Evaporation Rate: {evaporation_rate}, Average Best Value: {avg_score}")

            if avg_score > best_overall_value:
                best_overall_value = avg_score
                best_params = (alpha, beta, evaporation_rate)

print("Best Parameters:", best_params)
print("Best Overall Average Value:", best_overall_value)

Alpha: 0.5, Beta: 2, Evaporation Rate: 0.3, Average Best Value: 684.9
Alpha: 0.5, Beta: 2, Evaporation Rate: 0.6, Average Best Value: 684.1
Alpha: 0.5, Beta: 2, Evaporation Rate: 0.9, Average Best Value: 683.0
Alpha: 0.5, Beta: 3, Evaporation Rate: 0.3, Average Best Value: 684.5
Alpha: 0.5, Beta: 3, Evaporation Rate: 0.6, Average Best Value: 684.7
Alpha: 0.5, Beta: 3, Evaporation Rate: 0.9, Average Best Value: 684.8
Alpha: 0.5, Beta: 5, Evaporation Rate: 0.3, Average Best Value: 683.6
Alpha: 0.5, Beta: 5, Evaporation Rate: 0.6, Average Best Value: 681.9
Alpha: 0.5, Beta: 5, Evaporation Rate: 0.9, Average Best Value: 682.5
Alpha: 1, Beta: 2, Evaporation Rate: 0.3, Average Best Value: 681.7
Alpha: 1, Beta: 2, Evaporation Rate: 0.6, Average Best Value: 680.6
Alpha: 1, Beta: 2, Evaporation Rate: 0.9, Average Best Value: 679.7
Alpha: 1, Beta: 3, Evaporation Rate: 0.3, Average Best Value: 684.2
Alpha: 1, Beta: 3, Evaporation Rate: 0.6, Average Best Value: 682.9
Alpha: 1, Beta: 3, Evaporation

We choose to do a type of grid search in order to find the best parameters. 

Optimized parameters: The variance in scored is not significant enough to take specifics parameters over others.

In [26]:

class ACO:
    """
    Ant Colony Optimization (ACO) algorithm for solving the biscuit placement problem on a roll,
    considering defect constraints and maximizing value.

    Attributes:
        roll_length (int): Length of the roll.
        biscuits (list): List of Biscuit objects representing available biscuits.
        defects (dict): Mapping of roll positions to defect types and counts.
        num_ants (int): Number of ants used per iteration.
        num_iterations (int): Number of iterations to perform.
        alpha (float): Weight of pheromone influence in the probability calculation.
        beta (float): Weight of heuristic influence in the probability calculation.
        evaporation_rate (float): Rate at which pheromones decay after each iteration.
        pheromones (list): Pheromone levels for each biscuit and the skip option.
    """

    def __init__(self, roll_length, biscuits, defects, num_ants, num_iterations, alpha=1, beta=3, evaporation_rate=0.6):
        """
        Initializes the ACO algorithm with given parameters.

        Args:
            roll_length (int): Length of the roll.
            biscuits (list): List of Biscuit objects.
            defects (dict): Mapping of roll positions to defect types and counts.
            num_ants (int): Number of ants used per iteration.
            num_iterations (int): Number of iterations to perform.
            alpha (float): Weight of pheromone influence (default: 1).
            beta (float): Weight of heuristic influence (default: 3).
            evaporation_rate (float): Rate of pheromone decay (default: 0.6).
        """
        self.roll_length = roll_length
        self.biscuits = biscuits
        self.defects = defects
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.pheromones = [1] * (len(biscuits) + 1)  # Initialize pheromones for all biscuits and skip option

    def evaluate_solution(self, solution):
        """
        Evaluates the quality of a solution based on the total value and defect constraints.

        Args:
            solution (list): Sequence of biscuit IDs representing the solution.

        Returns:
            int: Total value of the solution after penalties.
        """
        total_value = 0
        current_position = 0

        for biscuit_id in solution:
            if biscuit_id == 0:  # Skip position
                total_value -= 1  # Penalize for skipping
                current_position += 1
                continue

            biscuit = self.biscuits[biscuit_id - 1]  # Adjust for 0-based indexing

            if current_position + biscuit.length > self.roll_length:
                break  # Biscuit doesn't fit, end evaluation

            # Count defects in the segment covered by this biscuit
            defects_in_segment = defaultdict(int)
            for pos in range(current_position, current_position + biscuit.length):
                for defect_type, count in self.defects[pos].items():
                    defects_in_segment[defect_type] += count

            # Check if biscuit can cover defects without exceeding its max tolerance
            if all(defects_in_segment[d] <= biscuit.max_defects.get(d, 0) for d in defects_in_segment):
                total_value += biscuit.value
                current_position += biscuit.length
            else:
                total_value -= 1  # Penalize if the biscuit cannot cover the defects

        return total_value

    def construct_solution(self):
        """
        Constructs a solution using probabilistic selection of biscuits based on pheromones and heuristics.

        Returns:
            list: Constructed solution as a sequence of biscuit IDs.
        """
        solution = []
        current_position = 0

        while current_position < self.roll_length:
            probabilities = []

            # Calculate selection probabilities for each biscuit
            for biscuit_id, biscuit in enumerate(self.biscuits):
                if current_position + biscuit.length > self.roll_length:
                    probabilities.append(0)  # Biscuit cannot fit
                    continue

                # Count defects in the segment covered by this biscuit
                defects_in_segment = defaultdict(int)
                for pos in range(current_position, current_position + biscuit.length):
                    for defect_type, count in self.defects[pos].items():
                        defects_in_segment[defect_type] += count

                # Determine if the biscuit is valid and calculate its probability
                if all(defects_in_segment[d] <= biscuit.max_defects.get(d, 0) for d in defects_in_segment):
                    heuristic = biscuit.value
                    pheromone = self.pheromones[biscuit_id]
                    probabilities.append((pheromone ** self.alpha) * (heuristic ** self.beta))
                else:
                    probabilities.append(0)  # Invalid biscuit

            # Add skipping as an option
            skip_pheromone = self.pheromones[len(self.biscuits)]
            probabilities.append((skip_pheromone ** self.alpha) * (1 ** self.beta))

            total_prob = sum(probabilities)
            if total_prob == 0:
                break  # No valid options, terminate

            probabilities = [p / total_prob for p in probabilities]
            selected_option = random.choices(range(len(self.biscuits) + 1), probabilities)[0]

            if selected_option == len(self.biscuits):  # Skip option
                solution.append(0)
                current_position += 1
            else:
                solution.append(selected_option + 1)  # Biscuit ID offset by 1
                current_position += self.biscuits[selected_option].length

        return solution

    def update_pheromones(self, solutions):
        """
        Updates pheromone levels based on the quality of solutions.

        Args:
            solutions (list): List of tuples (solution, value) from the current iteration.
        """
        # Evaporate pheromones
        self.pheromones = [p * (1 - self.evaporation_rate) for p in self.pheromones]

        # Reinforce pheromones based on solution values
        for solution, value in solutions:
            for biscuit_id in solution:
                if biscuit_id == 0:
                    self.pheromones[len(self.biscuits)] += value  # Update skip option pheromone
                else:
                    self.pheromones[biscuit_id - 1] += value

    def optimize(self):
        """
        Performs the optimization process over multiple iterations.

        Returns:
            tuple: Best solution and its value.
        """
        best_solution = None
        best_value = float('-inf')

        for _ in range(self.num_iterations):
            solutions = []

            # Generate solutions using all ants
            for _ in range(self.num_ants):
                solution = self.construct_solution()
                value = self.evaluate_solution(solution)
                solutions.append((solution, value))

                # Track the best solution
                if value > best_value:
                    best_solution = solution
                    best_value = value

            # Update pheromones based on solutions
            self.update_pheromones(solutions)

        return best_solution, best_value


# Grid Search
LENGTH = 500
NUM_ANTS = 25
NUM_ITERATIONS = 10

start=time.time()

aco = ACO(LENGTH, biscuits, defects, NUM_ANTS, NUM_ITERATIONS, 1, 3, 0.3)
best_solution,best_value = aco.optimize()
stop=time.time() -start
print("Best Overall Average Value:", best_value)
print("RunTime: ", stop)

Best Overall Average Value: 683
RunTime:  0.47302722930908203


# Results

| Method                             | Time (s) | Mean result | Best result |
|------------------------------------|----------|-------------|-------------|
| A*                                 | 2.66     | 706.126     | 715         |
| CSP                                | 0.30     | 715         | 715         |
| Dynamic programming                | 0.04     | 715         | 715         |
| Basic Genetic                      | 224.71   | ~470        | 479         |
| Genetic Sub segment                | 98.23    | ~640        | 644         |
| Genetic Sub segment & progressive  | 569.00   | ~670        | 675         |
| Ant colony                         | 0.47     | 684.9       | 691         |


## Results and Comparison of Algorithms

### 1. A* Algorithm
The A* algorithm was implemented with a heuristic designed to maximize the total value of placed cookies while minimizing unused areas on the biscuit roll. The approach excelled in producing optimal solutions within a constrained computational timeframe. Specifically:
- **Strengths**:
  - Highly effective at finding high-value placements.
  - Ensures optimality through an informed search guided by heuristics.
  - Efficient in exploring valid configurations, avoiding exhaustive search pitfalls.
- **Limitations**:
  - Computationally intensive for large-scale problems or when heuristics are poorly tuned.
  - Limited scalability without significant heuristic optimization.

### 2. Constraint Satisfaction Problem (CSP)
The CSP approach directly modeled the problem as variables and constraints, providing a straightforward framework for finding valid solutions.
- **Strengths**:
  - Efficiently eliminates invalid configurations, speeding up the search.
  - Naturally aligns with problems defined by strict constraints (e.g., no overlaps, defect thresholds).
  - Clear and logical implementation with minimal overhead.
- **Limitations**:
  - Limited flexibility in exploring innovative configurations.
  - May fail to optimize beyond the first valid solution without additional mechanisms.

### 3.Dynamic Programming (DP)
Dynamic Programming was employed to solve the problem by breaking it into overlapping subproblems. This approach focused on finding optimal placements using a table-driven methodology to store intermediate results.
- **Strengths**:
  - Guarantees optimal solutions by exhaustively exploring all subproblems.
  - Efficient in handling problems with a well-defined structure and clear recurrence relations.
  - Reduces redundant calculations through memoization or tabulation.
- **Limitations**:
  - Requires significant memory for large problem spaces due to the storage of intermediate states.
  - Limited flexibility in adapting to dynamic or highly complex constraints.

### 4.Genetic Algorithm (GA)
The Genetic Algorithm utilized elitism and progressive learning to optimize cookie placement. By breaking down the problem into smaller sub-problems, the algorithm improved its ability to learn effective solutions iteratively.
- **Strengths**:
  - Adapts well to varying roll sizes and cookie configurations.
  - Flexible and can integrate new constraints or objectives seamlessly.
- **Limitations**:
  - Struggled with maintaining solution diversity, particularly in later iterations.
  - Performance sensitive to parameter tuning (e.g., mutation rates, population size).
  - Hard to find rapidly converging solution while not being trapped in local optima.
  
### 5. Ant Colony Optimization (ACO)
Inspired by natural ant behaviors, ACO used pheromone levels and heuristic guidance to iteratively improve solutions. This adaptive approach balanced exploration and exploitation effectively.
- **Strengths**:
  - Robust in handling complex constraints and large solution spaces.
  - Adaptive parameter tuning (e.g., pheromone evaporation) improved solution quality over iterations.
  - Scalable to diverse problem scenarios with minimal modifications.
- **Limitations**:
  - Computationally intensive due to multiple iterations and agent simulations.
  - Sensitive to parameter settings and heuristic accuracy.



### Overall Comparison
| Algorithm         | Strengths                             | Limitations                          | Recommended Use Case               |
|-------------------|---------------------------------------|--------------------------------------|------------------------------------|
| A*                | Optimal solutions; efficient search   | High computational cost              | When precision and optimality are crucial |
| Genetic Algorithm | Flexible; scalable | Sensitive to diversity issues       | Large, complex problems requiring adaptability |
| CSP               | Fast; constraint-focused             | Limited exploration flexibility      | Problems with strict, well-defined constraints |
| ACO               | Adaptive; robust to complex scenarios | Computationally demanding            | Large-scale, multi-constraint optimization |
| Dynamic Programming | Optimal solutions; avoids redundancy | High memory usage                    | Problems with overlapping subproblems and clear structure |

### Feedback on Approaches
Each algorithm showcased unique advantages and trade-offs. While A* delivered precise and high-value solutions, it struggled with scalability for larger rolls. GA and ACO, by contrast, demonstrated adaptability and robustness in diverse scenarios but required careful parameter tuning and computational resources. CSP proved excellent for strict constraints but lacked the depth to explore innovative placements. DP excelled in solving structured problems optimally but faced limitations with memory requirements for larger scales.




# Conclusion

## Challenges Faced

### Implementation Complexity

- **Deciding on a Unified Design**: We spent considerable time determining how to structure the implementation in a way that would accommodate different search algorithms without requiring frequent changes. Ultimately, we settled on a straightforward approach using arrays and classes, which enabled efficient global management of elements. We decided to tweak the implementation a bit for the nature inspired algorithms to make it easier for them as they were computationally heavy. 

- **Handling Errors**: The handling of errors evolved significantly throughout the project. Initially, we experimented with several methods before deciding on rounding errors down to the nearest integer for simplicity. In the context of genetic algorithms, we implemented errors as a dictionary, allowing easier and more efficient data access.

### Algorithm Runtime

- **Performance Bottlenecks**: Due to the problem's vast number of possibilities and the large roll size, many algorithms proved too time-consuming to be practical. For instance, the A* algorithm was adapted to return only 500 results and terminate early instead of waiting for the entire frontier to be processed. We also leveraged external libraries like OR-Tools to enhance performance.

### Evaluating Results

- **Assessing Algorithm Effectiveness**: Since the maximum possible result wasn’t predefined, it was challenging to gauge an algorithm's success at first glance. To address this, we prioritized the A* algorithm initially for its structured approach.

- **Algorithm Limitations**: Some algorithms, such as genetic algorithms, struggled to find optimal solutions due to insufficient diversity. These algorithms often became trapped in local maxima, optimizing early but failing to explore alternative solutions with better potential in the long term.

## Future Directions

### Temporal Optimization

- **Improving Efficiency**: Algorithms like A* could benefit from more precise node management to reduce runtime without compromising accuracy.

### Achieving Optimal Results

- **Reaching the Maximum**: Certain algorithms were unable to reach the known best result (715). It would be worthwhile to experiment with more refined hyperparameter tuning and models architectures (GA) to see if achieving this maximum is feasible.

## Key Takeaways

This project provided a hands-on opportunity to explore and test various algorithms in a practical scenario, highlighting their respective strengths and weaknesses. Additionally, it required us to go beyond the classroom, experimenting with algorithms we hadn’t previously studied. Finally, implementing the project from the ground up taught us to approach problems with a long-term perspective, ensuring that our implementation could support diverse algorithmic needs effectively.

#### Recommendations:
- **Hybrid Models**: Combining A* for initial optimal placements with GA or ACO for iterative refinement could leverage the strengths of each approach.
- **Parameter Optimization**: Employ grid search or machine learning techniques to automate parameter tuning for GA and ACO.
- **Scenario-Specific Strategies**: Use CSP for tightly constrained problems, DP for structured, overlapping subproblem scenarios, and GA or ACO for open-ended, large-scale tasks.
